## Initialize Input Files for IPMOF (initialize_IPMOF.py)

### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cutOff radius and grid size for energy map

### Create input files for MOF files in a given directory
1. Reaf MOF files in ".mol2" format from given directory and create a list
2. Choose the first MOF1 as the stationary (map) MOF, initialize its variables, and calculate packed coordinates
3. For all the MOFs in the list:
4. Initialize MOF2 variables for the mobile MOF
5. Calculate energy map
6. Export energy map -> .json
7. Export MOF2 structure information -> .json

In [1]:
import math
import os
from energyMapFunctions import *

# Initialize force field parameters
excelFileDir = 'C:\\Users\\kutay\\iPython\\IPMOF\\FF_Parameters.xlsx'
UFF = readFFparameters(excelFileDir, 'UFF')
cutOff = 13

# Generate MOF list
fileDir = 'C:\\Kutay\\MOFs'
fileFormat = ".mol2"
MOFlist = generateMOFlist(fileDir, fileFormat)

MOFs = []
for MOFindex, mof in enumerate(MOFlist):
    MOFs.append(MOF())
    MOFs[MOFindex].dir = fileDir + '\\' + mof
    MOFs[MOFindex].file = open(MOFs[MOFindex].dir, 'r')
    MOFs[MOFindex].initialize()
    MOFs[MOFindex].initializeFF(UFF)
 
    # Calculate packing for base MOF
    packingFactor = Packing.factor(MOFs[MOFindex].UCsize, cutOff)
    MOFs[MOFindex].packingFactor = packingFactor
    translationVectors, UCvectors = Packing.vectors(packingFactor, MOFs[MOFindex].UCsize, MOFs[MOFindex].UCangle)
    MOFs[MOFindex].packedCoor = Packing.UC(translationVectors, packingFactor, UCvectors, MOFs[MOFindex].atomCoor)

from tabulate import tabulate

headers = ["MOF Name", "Atoms", "UCsize", "UCangle", "Packing Factor"]
table = [[] for mof in MOFs]

for MOFindex in range(len(MOFs)):
    MOFname = MOFlist[MOFindex].split('.')[0]
    MOFs[MOFindex]
    table[MOFindex].append(MOFname)
    table[MOFindex].append(MOFs[MOFindex].uniqueAtomNames)
    table[MOFindex].append(MOFs[MOFindex].UCsize)
    table[MOFindex].append(MOFs[MOFindex].UCangle)
    table[MOFindex].append(MOFs[MOFindex].packingFactor)

print(tabulate(table, headers))

MOF Name         Atoms                             UCsize                       UCangle                        Packing Factor
---------------  --------------------------------  ---------------------------  -----------------------------  ----------------
AgH6_CN2_3       ['H', 'N', 'Ag', 'C']             [6.5647, 6.5647, 11.347]     [99.8167, 99.8167, 102.913]    [5, 5, 5]
Cd2H26C17_NO2_6  ['H', 'Cd', 'N', 'C', 'O']        [12.6774, 12.6774, 18.0381]  [110.5733, 110.5733, 90.0001]  [5, 5, 3]
Cd2HC9NO8        ['O', 'Cd', 'H', 'C', 'N']        [8.3989, 8.535, 11.4883]     [89.325, 69.014, 67.269]       [5, 5, 5]
Co3H16_C9O7_2    ['Co', 'O', 'H', 'C']             [7.799, 9.212, 10.784]       [107.595, 92.877, 110.248]     [5, 5, 5]
CuNiH8C12_N2O_4  ['Cu', 'H', 'Ni', 'O', 'C', 'N']  [9.2189, 9.2189, 10.517]     [100.003, 100.003, 91.6964]    [5, 5, 5]
MgH2_CO2_2       ['Mg', 'O', 'H', 'C']             [9.8715, 18.1613, 18.2678]   [90.0, 90.0, 90.0]             [5, 3, 3]
MOF5             ['Z

In [4]:
# Creates atom list for non-repeating atoms from a list of MOF classes
atomList = getUniqueAtomList(MOFs)

# Creates atom list required in the energy map of a combination of MOFs
eMapAtomList = getEnergyMapAtomList(MOFs, atomList)

# Directory and format to export energy map and MOF structure files
exportFolder = "C:\\Users\\kutay\\iPython\\IPMOF\\initializeIPMOF"
exportFormat = ".js"

for MOFindex in range(len(MOFs)-11):  
        
    # Get MOF name for base MOF from the .mol2 file list
    MOFname = MOFlist[MOFindex].split('.')[0]
    
    # Create export directory for the base MOF and create folder
    exportMOFfolder = os.path.join(exportFolder, MOFname)
    os.mkdir(exportMOFfolder)
    
    # --------------------- Calculate Energy Map -------------------
    # For each MOF energy map is calculated with an atom list that consists of all the unique atoms 
    # for a given list of MOFs
    eMap = energyMap(MOFs[MOFindex], atomList, cutOff, 1)
    
    avgEnergyLimits = calculateEnergyLimits(MOFs, MOFlist, eMapAtomList, eMap)
    
    for MOFindex2 in range(len(MOFs)):
        
        # Get MOF name for mobile MOF from the .mol2 file list
        MOFname2 = MOFlist[MOFindex2].split('.')[0]
        
        # Create export directory for the mobile MOF and create folder
        exportMOFfolder2 = os.path.join(exportMOFfolder, MOFname2)
        os.mkdir(exportMOFfolder2)
        
        # Create export directory for energy map
        exportDir_eMap = os.path.join(exportMOFfolder2, MOFname + '+' + MOFname2 + '_eMap' + exportFormat)
        print(exportDir_eMap.split("\\")[-1] + ' is being exported')
        
        # Create export directory for mobile MOF
        exportDir_MOF = os.path.join(exportMOFfolder2, MOFname2 + exportFormat)
        print(exportDir_MOF.split("\\")[-1] + ' is being exported')

        #exportEnergyMapjs(eMap, atomList, exportDir_eMap)
        exportUniqueEnergyMapjs(eMap, avgEnergyLimits[MOFindex2], eMapAtomList, MOFindex, exportDir_eMap)

        # Export mobile MOF structure file
        exportMOFjs(MOFs[MOFindex2], exportDir_MOF)

MOF Name         Atoms                               Average Energy Limit
---------------  --------------------------------  ----------------------
AgH6_CN2_3       ['C', 'H', 'Ag', 'N']                        2.5061e+11
Cd2H26C17_NO2_6  ['C', 'Cd', 'O', 'H', 'N']                   7.88752e+06
Cd2HC9NO8        ['C', 'Cd', 'O', 'H', 'N']                   4.88113e+10
Co3H16_C9O7_2    ['C', 'O', 'H', 'Co']                        1.54375e+10
CuNiH8C12_N2O_4  ['Ni', 'C', 'O', 'H', 'Cu', 'N']             4.99012e+09
MgH2_CO2_2       ['Mg', 'C', 'O', 'H']                        1.47389e+07
MOF5             ['C', 'Zn', 'O', 'H']                   525309
NiH6_C7O3_2      ['Ni', 'C', 'O', 'H']                        3.06684e+09
YbH17C30_NO3_2   ['C', 'O', 'H', 'N', 'Yb']                   6.27484e+08
Zn2H22C27_NO_8   ['C', 'Zn', 'O', 'H', 'N']                   6.39647e+06
Zn4H12C24O13     ['C', 'Zn', 'O', 'H']                        8.75636e+08
Zn7_PO4_6        ['Zn', 'O', 'P']                

In [5]:
# Calculate Energy Map
# eMap = energyMap(MOF2, atomList, cutOff, 1)

#plotEnergyMap(eMap, 90, 0)
#plotPackedCell(MOF2.packedCoor, 90, 0)

# Create export directory for the energy map
#MOFname = MOFlist[MOFselection].split('.')[0]
#MOFname = 'MOFX'
#exportFolder = "C:\\Users\\kutay\\iPython\\IPMOF\\multipleMOF"
#exportFormat = ".js"
#exportDir_eMap = os.path.join(exportFolder, MOFname + '_eMap' + exportFormat)
#exportDir_MOF = os.path.join(exportFolder, MOFname + exportFormat)
#print(exportDir_eMap)
#print(exportDir_MOF)

#eMapDir = "C:\\Users\\kutay\\iPython\\IPMOF\\eMap.js"
#exportEnergyMapjs(eMap, atomList, exportDir_eMap)

#MOFexportDir = "C:\\Users\\kutay\\iPython\\IPMOF\\MOF.js"
#exportMOFjs(MOF2, exportDir_MOF)

print(exportDir_MOF.split("\\")[-1])

Zn7_PO4_6.js
